In [1]:
# this code uses the dataset that was cleaned for EDA
# note before starting
# Based on EDA there's weak linear relationships in this dataset
# Going off that the prediction is that LogisticRegression  will perform weakly
# And SVM will perform well
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

In [2]:
# Popularity tiers of size 20 will be used (same as in EDA report)
# Since cleaned dataset does not have nearly as big of a skew in popularity we will NOT be undersampling
#     class_weight = "balanced" will help deal with class imbalance
# only remove extreme outliers (duration less than  30 seconds or over 20 minutes, and tempo of 0)
#   keeping the rest helps keep natural variation - not noise
# Use PowerTransformer on heavily skewed features
# less heavily skewed features can use minmax scaling
# loudness, tempo, duration can use standard scaler
# OneHotEncoding for categorical variables

# load data
df = pd.read_csv('cleaned_dataset_EDA.csv')

# OUTLIERS
min_duration = 30 * 1000
max_duration = 20 * 60 * 1000

before = df.shape[0]
df = df[(df["duration_ms"] >= min_duration) & (df["duration_ms"] <= max_duration) & (df["tempo"] > 0)]
print(f"Removed {before - df.shape[0]} songs for duration/tempo outliers")

# Popularity bins
bins = list(range(0,101,20))
labels = [f"{i}-{i+20}" for i in range(0,100,20)]

df["popularity_tier"] = pd.cut(df["popularity"], bins=bins, labels=labels, right=False, include_lowest=True)

# rare case of row not having popularity tier, drop it
before = df.shape[0]
df = df.dropna(subset=["popularity_tier"])
print(f"Removed {before - df.shape[0]} rows with undefined popularity tier")

# target variable for classification is now popularity tier, should represent as categorical variable
y = df["popularity_tier"].astype(str)
print("Class distribution (popularity tiers):")
print(y.value_counts().sort_index())

# EDA showed that relationship between loudness and popularity varied by genre
# Logistic regression and SVM models do not automatically create or see these interactions
# So we will explicitly define some simple ones
# Lets use acoustic, heavy metal, and dance, as is used in the EDA
# For each genre, create a binary flag and multiply by loudness, allowing model to learn slopes for loudness for each of these genres
df["is_acoustic"] = (df["track_genre"] == "acoustic").astype(int)
df["is_heavymetal"] = (df["track_genre"] == "heavy-metal").astype(int)
df["is_dance"] = (df["track_genre"] == "dance").astype(int)

# interaction terms: multiply by loudness
df["loudness_x_acoustic"] = df["loudness"] * df["is_acoustic"]
df["loudness_x_heavymetal"] = df["loudness"] * df["is_heavymetal"]
df["loudness_x_dance"] = df["loudness"] * df["is_dance"]

# These columns will also be treated as numeric features
# speaking of...

# Define numeric and categorical features
numeric_features = [
"danceability", "energy", "loudness", "speechiness","acousticness",
"instrumentalness", "liveness", "valence", "tempo", "duration_ms",
"loudness_x_acoustic", "loudness_x_heavymetal", "loudness_x_dance",
]
categorical_features = ["track_genre", "key", "time_signature"]

# feature matrix
X = df[numeric_features + categorical_features].copy()
print("Feature matric shape before encoding + scaling: ",  X.shape)

Removed 228 songs for duration/tempo outliers
Removed 1 rows with undefined popularity tier
Class distribution (popularity tiers):
popularity_tier
0-20      18100
20-40     29955
40-60     25559
60-80      8668
80-100      563
Name: count, dtype: int64
Feature matric shape before encoding + scaling:  (82845, 16)


In [3]:
# Preprocessing transformer
# Logistic Regression and SVM are scale/distribution sensitive so if one feature has a larger distribution it can dominate the model
# Based on EDA some 0-1 features are heavily right-skewed (acousticness, instrumentalness, speechiness, liveness)
# some are less heavily skewed (danceability, energy, valence)
# some features are unbounded (loudness, tempo,  duration)
# Based on this, heavily skewed 0-1 features get powerTransformer to normalize the data and help models perform well
# less skewed 0-1 features get MinMaxScaler
# The others get StandardScaler to standardize unbounded features

skewed_features = ["acousticness", "instrumentalness", "speechiness", "liveness"]
less_skewed_features = ["danceability", "energy", "valence"]
# the interactions created are also unbounded
unbounded_features = ["loudness", "tempo", "duration_ms", "loudness_x_acoustic", "loudness_x_heavymetal", "loudness_x_dance"]

# instead of manually applying the different preprocessing steps to the different features
# we use columnTransformer
numeric_transformer = ColumnTransformer(
    transformers = [
        ("skewed_power", PowerTransformer(method="yeo-johnson"), skewed_features),
        ("bounded_minmax", MinMaxScaler(), less_skewed_features),
        ("unbounded_standard", StandardScaler(), unbounded_features)
    ],
    remainder="passthrough"
)

# For categorical features we use one hot encoding so models can use them properly
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

# full preprocessor
preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_transformer, numeric_features),
        ("categorical", categorical_transformer, categorical_features)
    ]
)

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

Train shape: (66276, 16), Test shape: (16569, 16)


In [4]:
##############################################################################
#                         Logistic Regression Model
##############################################################################

logistic_regression_pipeline = Pipeline(
    steps = [
        ("preprocessor", preprocessor),
        ("model", LogisticRegression(
            max_iter = 1000,
            class_weight = "balanced",
            random_state = 42,
            multi_class = "auto"
        ))
    ]
)

# Hyperparameter Grid for regression
log_reg_param_grid = {
    "model__C": [0.1, 1.0, 10.0],
    "model__penalty": ["l2"]
}

log_reg_grid = GridSearchCV(
    estimator = logistic_regression_pipeline,
    param_grid = log_reg_param_grid,
    scoring = "f1_macro",
    n_jobs = -1
)

print("Fitting Logistic Regression with cross-validation...")
log_reg_grid.fit(X_train, y_train)

print("Best Logistic Regression parameters:", log_reg_grid.best_params_)
print("Best Logistic Regression CV macro-F1:", log_reg_grid.best_score_)

# evaluate
y_pred_log = log_reg_grid.predict(X_test)

print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_log))

print("Logistic Regression Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_log))

Fitting Logistic Regression with cross-validation...


/Users/zoyamalik/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/zoyamalik/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/zoyamalik/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/zoyamalik/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1247: F

Best Logistic Regression parameters: {'model__C': 0.1, 'model__penalty': 'l2'}
Best Logistic Regression CV macro-F1: 0.47568397597682077
Logistic Regression Classification Report:
              precision    recall  f1-score   support

        0-20       0.70      0.63      0.66      3620
       20-40       0.72      0.60      0.66      5991
       40-60       0.65      0.53      0.58      5112
       60-80       0.27      0.46      0.34      1734
      80-100       0.06      0.72      0.12       112

    accuracy                           0.57     16569
   macro avg       0.48      0.59      0.47     16569
weighted avg       0.64      0.57      0.60     16569

Logistic Regression Confusion Matrix:
[[2266  381  283  305  385]
 [ 655 3593  902  747   94]
 [ 298  795 2714 1018  287]
 [  38  193  269  791  443]
 [   0    3    6   22   81]]


In [ ]:
##############################################################################
#                        Support Vector Machine Model
##############################################################################

svm_pipeline = Pipeline(
    steps = [
        ("preprocessor", preprocessor),
        ("model", SVC(
            kernel = "rbf",
            class_weight="balanced"
        ))
    ]
)

# Hyperparameter grid
# C controls overall strength of margin
# gamme controls how far the influence of each training example reaches
svm_param_grid = {
    "model__C": [0.1, 1.0, 10.0],
    "model__gamma": ["scale", 0.01, 0.1, 1.0]
}

svm_grid = GridSearchCV(
    estimator = svm_pipeline,
    param_grid = svm_param_grid,
    cv = 3,
    scoring = "f1_macro",
    n_jobs = -1
)

print("Fitting SVM with cross-validation...")
svm_grid.fit(X_train, y_train)

print("Best SVM parameters:", svm_grid.best_params_)
print("Best SVM CV macro-F1:", svm_grid.best_score_)


# Evaluate SVM on test set.
y_pred_svm = svm_grid.predict(X_test)


print("SVM — Classification Report (Test Set):")
print(classification_report(y_test, y_pred_svm))

print("SVM — Confusion Matrix (Test Set):")
print(confusion_matrix(y_test, y_pred_svm))

In [ ]:
def compute_metrics(y_true, y_pred, model_name):
    metrics = {
        "Model": model_name,
        "Accuracy": accuracy_score(y_true, y_pred),
        "Macro F1": f1_score(y_true, y_pred, average="macro"),
        "Weighted F1": f1_score(y_true, y_pred, average="weighted"),
        "Macro Precision": precision_score(y_true, y_pred, average="macro"),
        "Macro Recall": recall_score(y_true, y_pred, average="macro")
    }
    return metrics

log_metrics = compute_metrics(y_test, y_pred_log, "Logistic Regression")
svm_metrics = compute_metrics(y_test, y_pred_svm, "SVM (RBF Kernel)")

metrics_df = pd.DataFrame([log_metrics, svm_metrics])
metrics_df


In [ ]:
plt.figure(figsize=(10,6))
metrics_df.set_index("Model")[["Accuracy", "Macro F1", "Weighted F1"]].plot(kind="bar", figsize=(10,6))
plt.title("Overall Performance Comparison: Logistic Regression vs SVM")
plt.ylabel("Score")
plt.ylim(0, 1)
plt.xticks(rotation=0)
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()

In [ ]:
def plot_conf_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="viridis",
                xticklabels=sorted(y_test.unique()),
                yticklabels=sorted(y_test.unique()))
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.show()

plot_conf_matrix(y_test, y_pred_log, "Logistic Regression Confusion Matrix")
plot_conf_matrix(y_test, y_pred_svm, "SVM Confusion Matrix")

In [ ]:
def class_f1_scores(y_true, y_pred, model_name):
    report = classification_report(y_true, y_pred, output_dict=True)
    class_scores = {cls: report[cls]["f1-score"]
                    for cls in sorted(y_true.unique())}
    df = pd.DataFrame({"Class": class_scores.keys(),
                       f"{model_name} F1": class_scores.values()})
    return df

log_f1 = class_f1_scores(y_test, y_pred_log, "LogReg")
svm_f1 = class_f1_scores(y_test, y_pred_svm, "SVM")

f1_compare = log_f1.merge(svm_f1, on="Class")
f1_compare


In [ ]:
plt.figure(figsize=(10,6))
plt.plot(f1_compare["Class"], f1_compare["LogReg F1"], marker="o", label="Logistic Regression")
plt.plot(f1_compare["Class"], f1_compare["SVM F1"], marker="o", label="SVM")
plt.title("Class-wise F1 Score Comparison")
plt.ylabel("F1 Score")
plt.xlabel("Popularity Tier")
plt.ylim(0, 1)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## Resources

[1] PowerTransformer — scikit-learn 1.7.2 documentation. scikit-learn. Retrieved November 27, 2025 from https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html